In [223]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/koelectra/sent_action_multimodal.csv')
data

,video,big_sent,sm_sent,action
0,KETI_MULTIMODAL_0000000307,"['Happy', 'Rage', 'Happy', 'Sadness']","['Confident', 'Rage', 'Happy', 'Disappointed']",driving car
1,KETI_MULTIMODAL_0000000298,"['Happy', 'Rage', 'Hurt']","['Confident', 'Annoying', 'Hurt']",texting
2,KETI_MULTIMODAL_0000000308,"['Happy', 'Rage', 'Rage', 'Rage', 'Rage']","['Confident', 'Rage', 'Rage', 'Rage', 'Rage']",crying
3,KETI_MULTIMODAL_0000000299,"['Rage', 'Emberrassed', 'Rage', 'Hurt', 'Rage'...","['Rage', 'Guilty', 'Rage', 'Hurt', 'Rage', 'Co...",kissing
4,KETI_MULTIMODAL_0000000304,"['Happy', 'Rage', 'Rage', 'Rage', 'Rage']","['Happy', 'Rage', 'Annoying', 'Rage', 'Rage']",kissing
...,...,...,...,...
112,KETI_MULTIMODAL_0000000465,"['Rage', 'Rage', 'Rage', 'Rage', 'Rage', 'Sadn...","['Rage', 'Rage', 'Annoying', 'Annoying', 'Anno...",singing
113,KETI_MULTIMODAL_0000000464,"['Emberrassed', 'Happy', 'Rage', 'Anxiety', 'S...","['Guilty', 'Happy', 'Rage', 'Anxiety', 'Pessim...",texting
114,KETI_MULTIMODAL_0000000459,"['Rage', 'Rage', 'Rage', 'Happy', 'Rage']","['Rage', 'Rage', 'Rage', 'Happy', 'Rage']",moving furniture
115,KETI_MULTIMODAL_0000000460,"['Anxiety', 'Rage']","['Baffling', 'Annoying']",answering questions


In [225]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

In [226]:


def jaccard_similarity(data, question):

    union = set(data).union(set(question))
    intersection = set(data).intersection(set(question))

    jaccard_sim = len(intersection) / len(union)  

    return jaccard_sim

def jaccard_high(data, question, num, col):
    
    # data: 데이터프레임, question: 입력한 텍스트(질문), num: 자카드 유사도 상위 갯수
    data['jaccard_similarity'] = data[col].apply(lambda x: jaccard_similarity(x, question))

    return data[['video', 'big_sent', 'sm_sent', 'sent', 'action', 'jaccard_similarity']].sort_values(['jaccard_similarity'], ascending=False)[:num]

def tokenized_output(tokens):
    return tokens

def cos_similarity(data, question, col):

    tfidf_vectorizer = TfidfVectorizer(analyzer='word',
                                       tokenizer=tokenized_output,
                                       preprocessor=tokenized_output,
                                       token_pattern=None)
    
    tfidf_data = tfidf_vectorizer.fit_transform(data[col])
    tfidf_question = tfidf_vectorizer.transform([question])

    data['cosine_similarity'] = cosine_similarity(tfidf_data, tfidf_question).reshape(-1, )

    return data[['video', 'big_sent', 'sm_sent', 'sent', 'action','jaccard_similarity', 'cosine_similarity']].sort_values(['cosine_similarity'], ascending=False)

def video_rec(video_name, data):
  token_list = []
  for i in range(len(data)):
    bg_list = []
    for bg in data.loc[i, 'big_sent'][1:-1].replace("'", '').split(',') :
      bg_list.append(bg.strip())
    for sm in data.loc[i, 'sm_sent'][1:-1].replace("'", '').split(',') :
      bg_list.append(sm.strip())
    token_list.append(list(np.unique(np.array(bg_list))))
  data['sent'] = token_list
  ques_token = data.loc[data['video'] == video_name, 'sent'].values[0]
  result = jaccard_high(data, ques_token, 20, 'sent')
  result = cos_similarity(result, ques_token, 'sent')
  sent_cos_list = set(result.video[:20].values)

  ques_token = data.loc[data['video'] == video_name, 'action'].values
  result = jaccard_high(data, ques_token, 20, 'action')
  result = cos_similarity(result, ques_token, 'action')
  act_cos_list = set(result.video[:20].values)

  rec_list = list(sent_cos_list.intersection(act_cos_list))
  if len(rec_list) == 1:
    print('추천 비디오가 없습니다')
  else:
    try:
      return rec_list.remove(video_name)
    except:
      return rec_list

In [227]:
video_rec('KETI_MULTIMODAL_0000000304', data)


['KETI_MULTIMODAL_0000000408',
 'KETI_MULTIMODAL_0000000401',
 'KETI_MULTIMODAL_0000000422',
 'KETI_MULTIMODAL_0000000396']